# LLM Fundamentals with LangChain

## Using LLMs in LangChain

In [2]:
# LLMs

from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model='llama3.1')

model.invoke("The sky is")

'...blue! (or maybe grey, or cloudy, or sunny...) What were you going to say?'

In [3]:
# Chat Model

from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage

model = ChatOllama(model="llama3.1")
prompt = [HumanMessage("What is the capital of France?")]

model.invoke(prompt)

AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-24T14:31:15.3324601Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1180698400, 'load_duration': 109208800, 'prompt_eval_count': 17, 'prompt_eval_duration': 413545100, 'eval_count': 8, 'eval_duration': 650112000, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--85082919-006a-4d56-a4c4-baf469dcd87f-0', usage_metadata={'input_tokens': 17, 'output_tokens': 8, 'total_tokens': 25})

In [4]:
# Chat Model with SystemMessage interaction

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama.chat_models import ChatOllama

model = ChatOllama(model='llama3.1')
system_msg = SystemMessage(
    '''You are a helpful assistant that responds to questions with three exclamation marks.'''
)
human_msg = HumanMessage(
    '''What is the capital of France?'''
)

model.invoke([system_msg, human_msg])

AIMessage(content='Paris!!', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-24T14:31:16.8282216Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1469849700, 'load_duration': 93859400, 'prompt_eval_count': 37, 'prompt_eval_duration': 1172635500, 'eval_count': 3, 'eval_duration': 183269000, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--3eb70514-d595-405b-bc97-e1c83da684fe-0', usage_metadata={'input_tokens': 37, 'output_tokens': 3, 'total_tokens': 40})

## Making LLM Prompts Reusable

In [5]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import PromptTemplate

# both 'template' and 'model' can be reused many times

template = PromptTemplate.from_template(
    '''Answer the question based on the context below. 
    If the question cannot be answered using the information provided, answer "I don't know."

    Context: {context}
    
    Question: {question}

    Answer:'''
)

model = OllamaLLM(model='llama3.1')

# 'prompt' and 'completion' are the results of using template and model once

prompt = template.invoke({
    "context": """The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperforms their
        smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers
        can tap into tese models through Hugging Face's 'transformers' library, or by utilizing OpenAI and Cohere's offerings through the
        'openai' and 'cohere' libraries, respectively.""",
    "question":"Which model providers offer LLMs?"
})

completion = model.invoke(prompt)
completion

"OpenAI and Cohere offer LLMs through their respective libraries ('openai' and 'cohere')."

In [7]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# both 'template' and 'model' can be reused many times

template = ChatPromptTemplate.from_messages([
    ('system', '''Answer the question basen on the context below. If the question cannot be answered using hte information provided,
        answer with "I don't know".'''),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}')
])

model = ChatOllama(model='llama3.1')

# 'prompt' and 'completion' are the results of using template and model once

prompt = template.invoke({
    "context": """The most recent advancements in NLP are being driven by Large Language Models (LLMs). These models outperforms their
        smaller counterparts and have become invaluable for developers who are creating applications with NLP capabilities. Developers
        can tap into tese models through Hugging Face's 'transformers' library, or by utilizing OpenAI and Cohere's offerings through the
        'openai' and 'cohere' libraries, respectively.""",
    "question":"Which model providers offer LLMs?"
})

model.invoke(prompt)

AIMessage(content="Hugging Face ('transformers' library), OpenAI ('openai' library), and Cohere ('cohere' library) offer Large Language Models (LLMs).", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-10-24T14:41:04.4695804Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15346473400, 'load_duration': 6982090000, 'prompt_eval_count': 148, 'prompt_eval_duration': 5250743800, 'eval_count': 35, 'eval_duration': 3084087000, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--b4019ef5-6dd2-4b55-8640-634b0bee301b-0', usage_metadata={'input_tokens': 148, 'output_tokens': 35, 'total_tokens': 183})

## Getting Specific Formats out of LLMs